<a href="https://colab.research.google.com/github/steven1174/Python_Projects/blob/main/SQL/Human_Resources.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
%%capture

try:
    import duckdb
except:
    !pip install duckdb
    import duckdb

# Databases

In [3]:
urls = ['https://raw.githubusercontent.com/steven1174/Python_Projects/main/Datasets/HR_Departments.csv',
        'https://raw.githubusercontent.com/steven1174/Python_Projects/main/Datasets/HR_Employees.csv',
        'https://raw.githubusercontent.com/steven1174/Python_Projects/main/Datasets/HR_Jobs.csv',
        'https://raw.githubusercontent.com/steven1174/Python_Projects/main/Datasets/HR_Jobs_History.csv',
        'https://raw.githubusercontent.com/steven1174/Python_Projects/main/Datasets/HR_Locations.csv']

In [15]:
DEPARTMENTS = pd.read_csv(urls[0], header = None)
EMPLOYEES = pd.read_csv(urls[1], header = None)
JOBS = pd.read_csv(urls[2], header = None)
JOBS_HISTORY = pd.read_csv(urls[3], header = None)
LOCATIONS = pd.read_csv(urls[4], header = None)

In [16]:
DEPARTMENTS.columns = ['DEPT_ID_DEP','DEP_NAME','MANAGER_ID','LOC_ID']
EMPLOYEES.columns = ['EMP_ID','F_NAME','L_NAME','SSN','B_DATE','SEX','ADDRESS','JOB_ID','SALARY','MANAGER_ID','DEP_ID']
JOBS.columns = ['JOB_IDENT','JOB_TITLE','MIN_SALARY','MAX_SALARY']
JOBS_HISTORY.columns = ['EMPL_ID','START_DATE','JOBS_ID','DEPT_ID']
LOCATIONS.columns = ['LOCT_ID','DEP_ID_LOC']

In [17]:
con = duckdb.connect()

# Queries

<ul>
 <li>Query 1: Retrieve all employees whose address is in Elgin,IL.</li>
</ul>

In [20]:
query = """
	SELECT
			F_NAME,
			L_NAME,
			ADDRESS
	FROM EMPLOYEES
	WHERE ADDRESS LIKE '%Elgin,IL%' 
"""
df = con.execute(query).fetchdf()
df

,F_NAME,L_NAME,ADDRESS
0,Alice,James,"980 Berry ln, Elgin,IL"
1,Nancy,Allen,"111 Green Pl, Elgin,IL"
2,Ann,Jacob,"111 Britany Springs,Elgin,IL"


<ul>
 <li>Query 2: Retrieve all employees who were born during the 1970's.</li>
</ul>

In [21]:
query = """
	SELECT
			F_NAME,
			L_NAME,
      B_DATE,
      SEX	
	FROM EMPLOYEES
	WHERE B_DATE LIKE '%197%'
"""
df = con.execute(query).fetchdf()
df

,F_NAME,L_NAME,B_DATE,SEX
0,John,Thomas,01/09/1976,M
1,Alice,James,07/31/1972,F
2,Nancy,Allen,02/06/1978,F
3,Mary,Thomas,05/05/1975,F


<ul>
 <li>Query 3: Retrieve all employees in department 5 whose salary is between 60000 and 70000.</il>
</ul>

In [22]:
query = """
	SELECT
			*
	FROM EMPLOYEES
  WHERE 
    DEP_ID = 5 AND
    SALARY >= 60000 AND SALARY <= 70000
"""
df = con.execute(query).fetchdf()
df

,EMP_ID,F_NAME,L_NAME,SSN,B_DATE,SEX,ADDRESS,JOB_ID,SALARY,MANAGER_ID,DEP_ID
0,E1004,Santosh,Kumar,123459,07/20/1985,M,"511 Aurora Av, Aurora,IL",400,60000,30004,5
1,E1010,Ann,Jacob,123415,03/30/1982,F,"111 Britany Springs,Elgin,IL",220,70000,30004,5


In [23]:
query = """
	SELECT
			*
	FROM EMPLOYEES
  WHERE 
    DEP_ID = 5 AND
    SALARY BETWEEN 60000 AND 70000
"""
df = con.execute(query).fetchdf()
df

,EMP_ID,F_NAME,L_NAME,SSN,B_DATE,SEX,ADDRESS,JOB_ID,SALARY,MANAGER_ID,DEP_ID
0,E1004,Santosh,Kumar,123459,07/20/1985,M,"511 Aurora Av, Aurora,IL",400,60000,30004,5
1,E1010,Ann,Jacob,123415,03/30/1982,F,"111 Britany Springs,Elgin,IL",220,70000,30004,5


<ul>
 <li>Query 4A: Retrieve a list of employees ordered by department ID.</il>
</ul>

In [75]:
query = """
	SELECT
			*
	FROM EMPLOYEES
  ORDER BY DEP_ID
"""

df = con.execute(query).fetchdf()
df

,EMP_ID,F_NAME,L_NAME,SSN,B_DATE,SEX,ADDRESS,JOB_ID,SALARY,MANAGER_ID,DEP_ID
0,E1001,John,Thomas,123456,01/09/1976,M,"5631 Rice, OakPark,IL",100,100000,30001,2
1,E1005,Ahmed,Hussain,123410,01/04/1981,M,"216 Oak Tree, Geneva,IL",500,70000,30001,2
2,E1006,Nancy,Allen,123411,02/06/1978,F,"111 Green Pl, Elgin,IL",600,90000,30001,2
3,E1002,Alice,James,123457,07/31/1972,F,"980 Berry ln, Elgin,IL",200,80000,30002,5
4,E1003,Steve,Wells,123458,08/10/1980,M,"291 Springs, Gary,IL",300,50000,30002,5
5,E1004,Santosh,Kumar,123459,07/20/1985,M,"511 Aurora Av, Aurora,IL",400,60000,30004,5
6,E1010,Ann,Jacob,123415,03/30/1982,F,"111 Britany Springs,Elgin,IL",220,70000,30004,5
7,E1007,Mary,Thomas,123412,05/05/1975,F,"100 Rose Pl, Gary,IL",650,65000,30003,7
8,E1008,Bharath,Gupta,123413,05/06/1985,M,"145 Berry Ln, Naperville,IL",660,65000,30003,7
9,E1009,Andrea,Jones,123414,07/09/1990,F,"120 Fall Creek, Gary,IL",234,70000,30003,7


<ul>
 <li>Query 4B: Retrieve a list of employees ordered in descending order by department ID and within each department ordered alphabetically in descending order by last name.</il>
</ul>

In [76]:
query = """
	SELECT
			*
	FROM EMPLOYEES
  ORDER BY 11 DESC, 3 DESC
"""
df = con.execute(query).fetchdf()
df

,EMP_ID,F_NAME,L_NAME,SSN,B_DATE,SEX,ADDRESS,JOB_ID,SALARY,MANAGER_ID,DEP_ID
0,E1007,Mary,Thomas,123412,05/05/1975,F,"100 Rose Pl, Gary,IL",650,65000,30003,7
1,E1009,Andrea,Jones,123414,07/09/1990,F,"120 Fall Creek, Gary,IL",234,70000,30003,7
2,E1008,Bharath,Gupta,123413,05/06/1985,M,"145 Berry Ln, Naperville,IL",660,65000,30003,7
3,E1003,Steve,Wells,123458,08/10/1980,M,"291 Springs, Gary,IL",300,50000,30002,5
4,E1004,Santosh,Kumar,123459,07/20/1985,M,"511 Aurora Av, Aurora,IL",400,60000,30004,5
5,E1002,Alice,James,123457,07/31/1972,F,"980 Berry ln, Elgin,IL",200,80000,30002,5
6,E1010,Ann,Jacob,123415,03/30/1982,F,"111 Britany Springs,Elgin,IL",220,70000,30004,5
7,E1001,John,Thomas,123456,01/09/1976,M,"5631 Rice, OakPark,IL",100,100000,30001,2
8,E1005,Ahmed,Hussain,123410,01/04/1981,M,"216 Oak Tree, Geneva,IL",500,70000,30001,2
9,E1006,Nancy,Allen,123411,02/06/1978,F,"111 Green Pl, Elgin,IL",600,90000,30001,2


<ul>
 <li>Query 5A: For each department ID retrieve the number of employees in the department.</il>
</ul>

In [79]:
query = """
	SELECT
      DISTINCT(DEP_ID) dep_id,
      COUNT(DEP_ID) employees
	FROM EMPLOYEES
  GROUP BY 1
"""
df = con.execute(query).fetchdf()
df

,dep_id,employees
0,2,3
1,5,4
2,7,3


<ul>
 <li>Query 5B: For each department retrieve the number of employees in the department, and the average employees salary in the department.</il>
</ul>

In [80]:
query = """
	SELECT
      DISTINCT(DEP_ID) dep_id,
      COUNT(DEP_ID) employees,
      AVG(SALARY) avg_salary
	FROM EMPLOYEES
  GROUP BY 1
"""
df = con.execute(query).fetchdf()
df

,dep_id,employees,avg_salary
0,2,3,86666.666667
1,5,4,65000.000000
2,7,3,66666.666667


<ul>
 <li>Query 5C: In Query 5B limit the result to departments with fewer than 4 employees.</il>
</ul>

In [84]:
query = """
	SELECT
      DISTINCT(DEP_ID) dep_id,
      COUNT(DEP_ID) num_employees,
      AVG(SALARY) avg_salary
	FROM EMPLOYEES
  GROUP BY 1
  HAVING num_employees < 4
"""
df = con.execute(query).fetchdf()
df

,dep_id,num_employees,avg_salary
0,2,3,86666.666667
1,7,3,66666.666667
